In [6]:
import pandas as pd
import os
import numpy as np
import scipy

# PIL (Pillow) is a python image library that is used to handle operations on images
# Image is an object representation of an Image for manipulation
# ImageEnhance controls the brightness of the image
# ImageChops (Chanel Operations) is a module that is used to apply arithmetic operations to images
from PIL import Image, ImageEnhance, ImageChops

import matplotlib.pyplot as plt
import pickle
import warnings
import random
from shutil import copy

In [2]:
from tensorflow.keras.models import load_model

In [3]:
# preprocess the image data
# the preprocessing step creates ELA images so that we filter out the unwanted file formats
# ELA aka Error Level Analysis is a forensic technique that is used to analyze images by comparing
# them through different levels of compression.
def preProcessImages(imagesPath, file, resavePath):
    
    imagePath = os.path.join(imagesPath, file)
        
    #we accept 3 lossy file extensions: jpg, jpeg, and png. Process images in this formar only
    if file.endswith('jpg') or file.endswith('jpeg') or file.endswith('png'):
            
        #open the image and convert it to RGB mode
        image = Image.open(imagePath).convert('RGB')
            
        #resave the image with the new mode and as a jpg
        # when we save an image with a quality of 90, that means we store the image
        # with high quality but moderate compression
        image.save('resavedImage.jpg', 'JPEG', quality=90)
            
        # reopen the resaved image
        resavedImage = Image.open('resavedImage.jpg')
            
        # Calculate the ELA by taking the difference between the originial and the resaved image
        elaImage = ImageChops.difference(image, resavedImage)
            
        # get the maximum pixel value
        maxPixelValue = max([val[1] for val in elaImage.getextrema()])
            
        # if the max value is 0, set it to 1 so that we do not divide by 0
        if(maxPixelValue <= 0):
            maxPixelValue = 1
                
        # using the max value, scale the pixels to the range of [0,255]
        scaleVal = 255.0/maxPixelValue
            
        #using the scaled value, enhance the image and save it
        elaImage = ImageEnhance.Brightness(elaImage).enhance(scaleVal)
            
        #save the image in the reserved file path as a jpeg
        elaImage.save(os.path.join(resavePath, os.path.basename(imagePath)), 'JPEG')
        
        # Resize the image to the input size expected by the model
        elaImage = elaImage.resize((256, 256))  # Assuming InceptionV3 expects input size of 299x299

        # Convert the image to a numpy array
        imgArray = np.asarray(elaImage)

        # Reshape the array to match the expected input shape of the model
        #imgArray = np.expand_dims(img_array, axis=0)
        imgArray = imgArray[np.newaxis, :]

        return imgArray

In [4]:
model = load_model('imageTemperingModel.h5')
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 inception_v3 (Functional)   (None, 6, 6, 2048)           2180278   ['input_3[0][0]']             
                                                          4                                       
                                                                                                  
 global_average_pooling2d (  (None, 2048)                 0         ['inception_v3[0][0]']        
 GlobalAveragePooling2D)                                                                          
                                                                                              

In [7]:
sampleImagePath = '/Users/avinashgupta/Documents/Personal_Projects/MachineLearning-Projects/ImageTamperingDetection/data'
sampleImage = 'yuleLog.jpg'
resultPath = '/Users/avinashgupta/Documents/Personal_Projects/MachineLearning-Projects/ImageTamperingDetection/'

resultImageArray = preProcessImages(sampleImagePath, sampleImage, resultPath)

In [8]:
print(resultImageArray.shape)

(1, 256, 256, 3)


In [10]:
predictions = model.predict(resultImageArray)

if predictions[0] < 0.5:
    print("Image is authentic")
else:
    print("Image is tempered")

1/1 [==============================] - 1s 587ms/step
Image is authentic
